CVOPXT FORM:
$$min \frac{1}{2}λ^TPλ - q^Tλ$$
$$s.t \space Gx \le h \\ Ax=b$$ 
Same as:
- $α_i \ge 0$
- $\sum\alpha_iy_i = 0$
- $P = V^TV$
- $V = [y_1x_1, y_2x_2, ..., y_nx_n]$

In [1]:
from cvxopt import matrix, solvers
import numpy as np

def linear(x, z):
  return np.dot(z, x.T)

def gaussian(x, z, sigma=0.1):
  return np.exp(-np.linalg.norm(x-z, axis=1)**2 / (2*sigma**2)) #not this

def polynomial(x, z, p=6):
    return (1 + np.dot(x, z.T)) ** p

class SVM():
  def __init__(self, kernel=linear, C=1e5):
    self.kernel = kernel
    self.C = C
  
  def fit(self, X, y):
    #Calculate kernel:
    m = X.shape[0]
    K = np.zeros((m, m))
    for i in np.arange(m):
        K[:, i] = self.kernel(X[i, :], X).flatten()
    #Calculate alpha via CVXOPT:
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones((m ,1)))
    G = matrix(np.vstack((-1 * np.eye(m), np.eye(m))))
    h = matrix(np.hstack((np.zeros((m)), self.C * np.ones(m))))
    A = matrix(y, (1, m), 'd')
    b = matrix(np.zeros((1)))

    solvers.options["show_progress"] = False
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    sv = np.argwhere((alphas > 1e-3) * (alphas < self.C))[:, 0]
    self.alphas = alphas[sv]
    self.X = X[sv]
    self.y = y[sv]
    self.K = K[:, sv][sv, :]
    self.w = np.sum(self.X * self.y * self.alphas, axis=0)
    self.b = np.mean(self.y - np.sum(self.alphas * self.y * self.K, axis=1)[:, None])

  def predict(self, X):
    """
    Binary classification for class X
    """
    #index_spv = np.where(np.logical_and(alphas > 1e-6, alphas < self.C))[0]
    y_predict = np.zeros((X.shape[0]))
    #print("alphas: ", self.alphas)
    #print("y: ", self.y)
    for i in np.arange(X.shape[0]):
      y_predict[i] = np.sum(self.alphas * self.y * self.kernel(X[i], self.X))
    return np.sign(y_predict + self.b)    
    
  
  def get_index_support_vector(self):
    return self.index_spv

ModuleNotFoundError: No module named 'cvxopt'